In [39]:
import os
#import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from download_imdb import maybe_download_imdb
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

In [2]:
stop_string = """
's
a
able
about
above
abst
accordance
according
accordingly
across
act
actually
added
adj
affected
affecting
affects
after
afterwards
again
against
ah
all
almost
alone
along
already
also
although
always
am
among
amongst
an
and
announce
another
any
anybody
anyhow
anymore
anyone
anything
anyway
anyways
anywhere
apparently
approximately
are
aren
arent
arise
around
as
aside
ask
asking
at
auth
available
away
awfully
back
be
became
because
become
becomes
becoming
been
before
beforehand
begin
beginning
beginnings
begins
behind
being
believe
below
beside
besides
between
beyond
biol
both
brief
briefly
but
by
came
can
cannot
can't
cause
causes
certain
certainly
co
com
come
comes
contain
containing
contains
could
couldnt
date
did
didn't
different
do
does
doesn't
doing
done
don't
down
downwards
due
during
each
ed
edu
effect
eg
eight
eighty
either
else
elsewhere
end
ending
enough
especially
et
et-al
etc
even
ever
every
everybody
everyone
everything
everywhere
ex
except
far
few
fifth
first
five
fix
followed
following
follows
for
former
formerly
forth
found
four
from
further
furthermore
gave
get
gets
getting
give
given
gives
giving
go
goes
gone
got
gotten
had
happens
hardly
has
hasn't
have
haven't
having
he
hed
hence
her
here
hereafter
hereby
herein
heres
hereupon
hers
herself
hes
hi
hid
him
himself
his
hither
home
how
howbeit
however
hundred
i
id
ie
if
i'll
im
immediate
immediately
importance
important
in
inc
indeed
index
information
instead
into
invention
inward
is
isn't
it
itd
it'll
its
itself
i've
just
keep
keeps
kept
kg
km
know
known
knows
largely
last
lately
later
latter
latterly
least
less
lest
let
lets
like
liked
likely
line
little
'll
look
looking
looks
ltd
made
mainly
make
makes
many
may
maybe
me
mean
means
meantime
meanwhile
merely
mg
might
million
miss
ml
more
moreover
most
mostly
mr
mrs
much
mug
must
my
myself
name
namely
nay
near
nearly
necessarily
necessary
need
needs
neither
never
nevertheless
new
next
nine
ninety
no
nobody
non
none
nonetheless
noone
nor
normally
nos
not
n't
noted
nothing
now
nowhere
obtain
obtained
obviously
of
off
often
oh
ok
okay
old
omitted
on
once
one
ones
only
onto
or
ord
other
others
otherwise
ought
our
ours
ourselves
out
outside
over
overall
owing
own
p
page
pages
part
particular
particularly
past
per
perhaps
placed
please
plus
poorly
possible
possibly
potentially
predominantly
present
previously
primarily
probably
promptly
proud
provides
put
que
quickly
quite
ran
rather
readily
really
recent
recently
ref
refs
regarding
regardless
regards
related
relatively
research
respectively
resulted
resulting
results
right
run
said
same
saw
say
saying
says
sec
section
see
seeing
seem
seemed
seeming
seems
seen
self
selves
sent
seven
several
shall
she
shed
she'll
shes
should
shouldn't
show
showed
shown
showns
shows
significant
significantly
similar
similarly
since
six
slightly
so
some
somebody
somehow
someone
somethan
something
sometime
sometimes
somewhat
somewhere
soon
sorry
specifically
specified
specify
specifying
still
stop
strongly
sub
substantially
successfully
such
sufficiently
suggest
sup
sure
"""

In [3]:
stops = set(stop_string.strip().splitlines())

In [4]:
def clean_y(y_raw):
    """
    Strips out True and False values from the dicts in y_raw. Makes and returns an np array of dtype=int from the values
    """
    y_clean = np.array([item["POSITIVE"] for item in y_raw], dtype=int)
    return y_clean


filename = 'imdb_thinc_data.pickle'
filepath = os.path.join('..','data','thinc',filename)
_, _, X_test_raw, y_test_raw =maybe_download_imdb(filepath)


In [22]:
X_array_raw= list(X_test_raw[:50])
y_array_raw= list(y_test_raw[:50])
X_array_raw

['This film is exactly what its title describes--an attempt to get you to buy into what the writers have to offer.\n\n\n\nFirst, it\'s kinda fun to see the 1996-style Toronto I remember with all its silly haircuts, sunglasses, clothes, and attitude. It really hasn\'t changed any; just a nice, safe, cheap, provincial little urban backwater that makes a great meeting place for international film types! It\'s also amusing to see Kenny and Spenny head to L.A. and find out that it\'s Toronto all over again, only with a strange assortment of beach bums, musicians, fortune tellers, and yet more uppity film types.\n\n\n\nI don\'t see Pitch as a film to be enjoyed; it\'s not entertainment unless the viewer enjoys watching someone\'s aspirations being trampled. I take Pitch as a warning that power and money is really held by studio execs and production houses. Would-be (and "successful") writers, musicians, and actors are still mere transients even when they reach the Big Time.\n\n\n\nSo, Kenny 

In [23]:
type(X_array_raw), type(X_array_raw[0])

(list, str)

In [ ]:
#use this as supplied tokenizer
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

In [42]:
vec = CountVectorizer(ngram_range=(1, 1), 
                      stop_words=stops, 
                      strip_accents='unicode',
                     token_pattern=r'\b[a-zA-Z]{3,}\b',
                     tokenizer=LemmaTokenizer())

counts = vec.fit_transform(X_array_raw)

In [43]:
len(vec.get_feature_names())

2754

In [44]:
transformer = TfidfTransformer()
X_tfidf = transformer.fit_transform(counts)
y_array = clean_y(y_array_raw)

In [68]:
param_grid={'C':[1e-3,1e-2,1e-1,1,2,5,10,15,20,30,40,50,75,100,1000,10000]}

In [69]:
logreg = LogisticRegression(random_state=42)
c_validator = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [70]:
grid = GridSearchCV(logreg, n_jobs=-1, param_grid=param_grid,cv=c_validator )

In [71]:
grid.fit(X_tfidf,y_array)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 2, 5, 10, 15, 20, 30, 40, 50, 75, 100, 1000, 10000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [72]:
logreg = grid.best_estimator_

In [73]:
y_pred = logreg.predict(X_tfidf)
y_pred

array([1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 1])

In [74]:
logreg.score(X_tfidf,y_array)

1.0

In [75]:
logreg.get_params()

{'C': 20,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'ovr',
 'n_jobs': 1,
 'penalty': 'l2',
 'random_state': 42,
 'solver': 'liblinear',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [76]:
y_pred = logreg.predict(X_tfidf)

In [77]:
confusion_matrix(y_array, y_pred)

array([[27,  0],
       [ 0, 23]])